In [ ]:
"""
Spotify Metadata Updater for FLAC/MP3/WAV files
- Organized into sections: config, music_utils, search_utils, spotify_client, main
- English comments and improved logging
- WAV support: detects WAV with ID3 chunk vs WAV with RIFF INFO; uses whichever exists
- Filename fallback: if no "Artist - Title" separator, treat whole stem as title (so title-only searches work)
- Behavior: edit a TEMP copy, send original to trash, move temp into original path (original preserved in trash)
"""

# -------------------------
# CONFIG SECTION
# -------------------------
from pathlib import Path
import shutil
import logging
import time
import base64
import json
import re
from typing import Optional, Tuple, List
import requests
from mutagen.flac import FLAC, Picture
from mutagen.id3 import ID3, APIC, TIT2, TPE1, TALB, TDRC, TRCK, TPOS, TCON, ID3NoHeaderError
from mutagen.wave import WAVE
import unicodedata
from send2trash import send2trash
import platform

# User-editable configuration
CREDENTIALS_PATH = Path("credentials.json")
RECURSIVE = False
PROCESS_TOP_X = 10000

# Spotify endpoints
SPOTIFY_TOKEN_URL =         "https://accounts.spotify.com/api/token"
SPOTIFY_SEARCH_URL =        "https://api.spotify.com/v1/search"
SPOTIFY_ARTIST_URL =        "https://api.spotify.com/v1/artists/{}"
SPOTIFY_ARTIST_ALBUMS_URL = "https://api.spotify.com/v1/artists/{}/albums"
SPOTIFY_ALBUM_TRACKS_URL =  "https://api.spotify.com/v1/albums/{}/tracks"

# Timeouts and limits
REQUEST_TIMEOUT = 12
SPOTIFY_MAX_LIMIT = 50

# Behavior flags
OVERWRITE_TITLE_ARTIST_OR_ALBUM = 1   # 0 = preserve title/artist/album, 1 = overwrite
UPDATE_ONLY_GENRE = 1                 # 1 = only update genre
PRINT_SEARCH_INFO = 1                 # 1 = extended logs
SEARCH_CANDIDATE_LIMIT = 5            # number of spotify tracks to search per music file
MARKET: Optional[str] = None          # set e.g. "US" or "ES" to restrict results

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")


# -------------------------
# MUSIC UTILITIES SECTION
# -------------------------
# Small helpers for filename parsing, tag reading/writing, temp file handling, and images.
_FILENAME_SPLIT_RE = re.compile(r"\s[-–—]\s")


def infer_artist_title_from_filename(p: Path) -> Tuple[Optional[str], Optional[str]]:
    """
    Infer artist and title from filename.
    - If filename contains "Artist - Title" (separator - or long dashes), return (artist, title).
    - If filename contains a single hyphen, split on first hyphen.
    - If filename has no separator, treat the entire stem as TITLE (artist unknown).
      This is important because many WAV files are just 'Title.wav'.
    """
    stem = p.stem.strip()
    if not stem:
        return None, None
    m = _FILENAME_SPLIT_RE.split(stem, maxsplit=1)
    if len(m) == 2:
        artist = m[0].strip()
        title = m[1].strip()
        return (artist or None), (title or None)
    # fallback: if there's a simple '-' separator (without spaces)
    if "-" in stem:
        parts = stem.split("-", 1)
        artist = parts[0].strip()
        title = parts[1].strip()
        # if the left side looks like a single word and right side empty, still accept title
        return (artist or None), (title or None)
    # NO separator: treat whole stem as title (this is the change that fixes your 'Blame Myself.wav' case)
    return None, stem or None


def unique_temp_copy(src: Path) -> Path:
    """
    Create a unique temporary copy filename next to the source.
    Returns the temp path.
    """
    base_tmp = src.name + ".tmp"
    temp_path = src.with_name(base_tmp)
    i = 0
    while temp_path.exists():
        i += 1
        temp_path = src.with_name(f"{src.name}.tmp{i}")
    shutil.copy2(str(src), str(temp_path))
    return temp_path


def send_original_to_trash(original: Path) -> None:
    """
    Attempt to send original to trash; if fails, unlink.
    """
    try:
        send2trash(str(original))
    except Exception:
        try:
            original.unlink()
        except Exception:
            pass


# -------------------------
# SEARCH UTILITIES SECTION
# -------------------------
# Normalization and token containment logic used to compare Spotify candidates deterministically.

def _strip_parentheses_with_feat(s: Optional[str]) -> str:
    """
    Remove (feat ...) or [feat ...] blocks while preserving other parentheses content.
    """
    if not s:
        return ""
    def repl(m):
        inner = m.group(1)
        if re.search(r"\b(feat\.?|ft\.?)\b", inner, flags=re.IGNORECASE):
            return " "
        return m.group(0)
    s = re.sub(r"\(([^)]*)\)", repl, s)
    s = re.sub(r"\[([^]]*)\]", repl, s)
    s = re.sub(r"\s+", " ", s).strip()
    return s


def _extract_remixer_tokens_from_title(s: Optional[str]) -> List[str]:
    """
    Extract tokens from '(XXX Remix)' or '[XXX Remix]' blocks for matching remixers.
    """
    if not s:
        return []
    res: List[str] = []
    for m in re.finditer(r"\(([^)]*remix[^)]*)\)", s, flags=re.IGNORECASE):
        inner = m.group(1)
        name = re.sub(r"\bremix\b", " ", inner, flags=re.IGNORECASE)
        name = re.sub(r"[^0-9a-zA-Z\s]", " ", name)
        name = unicodedata.normalize("NFKD", name)
        name = "".join(ch for ch in name if not unicodedata.combining(ch))
        name = re.sub(r"\s+", " ", name).strip().lower()
        if name:
            res.extend([t for t in name.split() if t])
    for m in re.finditer(r"\[([^]]*remix[^]]*)\]", s, flags=re.IGNORECASE):
        inner = m.group(1)
        name = re.sub(r"\bremix\b", " ", inner, flags=re.IGNORECASE)
        name = re.sub(r"[^0-9a-zA-Z\s]", " ", name)
        name = unicodedata.normalize("NFKD", name)
        name = "".join(ch for ch in name if not unicodedata.combining(ch))
        name = re.sub(r"\s+", " ", name).strip().lower()
        if name:
            res.extend([t for t in name.split() if t])
    return res


def _normalize_text_basic(s: Optional[str]) -> str:
    """
    Light normalization: remove diacritics, lowercase, remove parentheses characters,
    strip 'feat' tokens and non-alphanumeric characters except spaces.
    """
    if not s:
        return ""
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[()]+", " ", s)
    s = re.sub(r"\b(feat\.?|ft\.?)\b", " ", s, flags=re.IGNORECASE)
    s = re.sub(r"[^0-9a-zA-Z\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip().lower()
    return s


def _normalize_artist_for_search(s: Optional[str]) -> str:
    if not s:
        return ""
    s2 = s.replace(",", " ").replace("\\", " ").replace("/", " ")
    s2 = _strip_parentheses_with_feat(s2)
    return _normalize_text_basic(s2)


def _normalize_title_for_search(s: Optional[str]) -> str:
    if not s:
        return ""
    s2 = _strip_parentheses_with_feat(s)
    return _normalize_text_basic(s2)


def _tokens(n: str) -> List[str]:
    if not n:
        return []
    return [t for t in n.split() if t]


def _tokens_in_candidate(tokens: List[str], candidate_norm: str) -> bool:
    """
    All tokens must be present as whole words in candidate normalized string.
    """
    if not tokens:
        return True
    cand_set = set(candidate_norm.split())
    return all(tok in cand_set for tok in tokens)


def _build_sanitized_query(n_artist: str, n_title: str, n_album: str, fielded: bool = True) -> str:
    """
    Build a Spotify query. If fielded, uses track:"..." artist:"..." album:"...".
    Otherwise return plain sanitized concatenation.
    """
    def quote_and_escape(s: str) -> str:
        s2 = s.replace('"', ' ')
        s2 = re.sub(r'\s+', ' ', s2).strip()
        return f'"{s2}"' if s2 else ''
    if fielded and (n_artist or n_title or n_album):
        parts = []
        if n_title:
            parts.append(f'track:{quote_and_escape(n_title)}')
        if n_artist:
            parts.append(f'artist:{quote_and_escape(n_artist)}')
        if n_album:
            parts.append(f'album:{quote_and_escape(n_album)}')
        return " ".join([p for p in parts if p])
    parts = []
    if n_artist:
        parts.append(n_artist)
    if n_title:
        parts.append(n_title)
    if n_album:
        parts.append(n_album)
    return " ".join(parts) if parts else '""'


# -------------------------
# SPOTIFY CLIENT / SEARCH SECTION
# -------------------------
# Token retrieval + search functions + paginated candidate evaluation. This is the same
# robust containment matching you had but now included fully here in the module.

def get_spotify_token(client_id: str, client_secret: str, ttl_margin: int = 5) -> Tuple[str, int]:
    """
    Obtain a client-credentials token from Spotify.
    Returns (token, expires_at_epoch_seconds).
    """
    auth = base64.b64encode(f"{client_id}:{client_secret}".encode("utf-8")).decode("ascii")
    headers = {"Authorization": f"Basic {auth}"}
    data = {"grant_type": "client_credentials"}
    resp = requests.post(SPOTIFY_TOKEN_URL, headers=headers, data=data, timeout=REQUEST_TIMEOUT)
    resp.raise_for_status()
    j = resp.json()
    token = j["access_token"]
    expires_in = int(j.get("expires_in", 3600))
    expires_at = int(time.time()) + expires_in - ttl_margin
    return token, expires_at


def spotifysearch(token: str, q: str, type_: str = "track", limit: int = 20, offset: int = 0, market: Optional[str] = None) -> Optional[dict]:
    """
    Basic wrapper for the Spotify search endpoint.
    Returns parsed JSON or None on failure.
    """
    headers = {"Authorization": f"Bearer {token}"}
    params = {"q": q, "type": type_, "limit": limit, "offset": offset}
    if market:
        params["market"] = market
    try:
        r = requests.get(SPOTIFY_SEARCH_URL, headers=headers, params=params, timeout=REQUEST_TIMEOUT)
    except Exception:
        return None
    if r.status_code == 401 or not r.ok:
        return None
    try:
        return r.json()
    except Exception:
        return None


def spotify_get_artist_albums(token: str, artist_id: str, limit: int = SPOTIFY_MAX_LIMIT, offset: int = 0, market: Optional[str] = None) -> Optional[dict]:
    headers = {"Authorization": f"Bearer {token}"}
    params = {"limit": limit, "offset": offset}
    if market:
        params["market"] = market
    try:
        r = requests.get(SPOTIFY_ARTIST_ALBUMS_URL.format(artist_id), headers=headers, params=params, timeout=REQUEST_TIMEOUT)
    except Exception:
        return None
    if not r.ok:
        return None
    try:
        return r.json()
    except Exception:
        return None


def spotify_get_album_tracks(token: str, album_id: str, limit: int = SPOTIFY_MAX_LIMIT, offset: int = 0, market: Optional[str] = None) -> Optional[dict]:
    headers = {"Authorization": f"Bearer {token}"}
    params = {"limit": limit, "offset": offset}
    if market:
        params["market"] = market
    try:
        r = requests.get(SPOTIFY_ALBUM_TRACKS_URL.format(album_id), headers=headers, params=params, timeout=REQUEST_TIMEOUT)
    except Exception:
        return None
    if not r.ok:
        return None
    try:
        return r.json()
    except Exception:
        return None


def spotify_find_best_match(token: str, artist: Optional[str], album: Optional[str], title: Optional[str],
                            combined_limit: int = None) -> Optional[dict]:
    """
    Containment-based search across several fielded and non-fielded queries.
    - Collect unique candidates (dedup by Spotify id or normalized key).
    - Evaluate containment: require tokens from title/artist/album to be present.
    - Return first accepted candidate object, or None.
    """
    if combined_limit is None:
        combined_limit = SEARCH_CANDIDATE_LIMIT

    n_artist = _normalize_artist_for_search(artist) if artist else ""
    n_title = _normalize_title_for_search(title) if title else ""
    n_album = _normalize_title_for_search(album) if album else ""

    artist_tokens = _tokens(n_artist)
    title_tokens = _tokens(n_title)
    album_tokens = _tokens(n_album)
    remixer_tokens = _extract_remixer_tokens_from_title(title or "")

    if PRINT_SEARCH_INFO:
        logging.info("Sanitized search input: artist='%s' | title='%s' | album='%s'", n_artist, n_title, n_album)

    # Build queries: fielded first, then plain
    queries: List[Tuple[str, str]] = []
    primary_q_fielded = _build_sanitized_query(n_artist, n_title, n_album, fielded=True)
    if primary_q_fielded:
        queries.append(("track", primary_q_fielded))
    at_q_fielded = _build_sanitized_query(n_artist, n_title, "", fielded=True)
    if at_q_fielded and at_q_fielded != primary_q_fielded:
        queries.append(("track", at_q_fielded))
    aa_q_fielded = _build_sanitized_query(n_artist, "", n_album, fielded=True)
    if aa_q_fielded and aa_q_fielded not in (primary_q_fielded, at_q_fielded):
        queries.append(("album", aa_q_fielded))
    t_q_fielded = _build_sanitized_query("", n_title, "", fielded=True)
    if t_q_fielded and t_q_fielded not in (primary_q_fielded, at_q_fielded, aa_q_fielded):
        queries.append(("track", t_q_fielded))
    a_q_fielded = _build_sanitized_query("", "", n_album, fielded=True)
    if a_q_fielded and a_q_fielded not in (primary_q_fielded, at_q_fielded, aa_q_fielded, t_q_fielded):
        queries.append(("album", a_q_fielded))

    primary_q_plain = _build_sanitized_query(n_artist, n_title, n_album, fielded=False)
    if primary_q_plain and primary_q_plain not in (q for _, q in queries):
        queries.append(("track", primary_q_plain))

    seen_keys = set()
    overall_idx = 0

    for (kind, q) in queries:
        if PRINT_SEARCH_INFO:
            logging.info("Query base: '%s' | type=%s | target=%d", q, kind, combined_limit)
        offset = 0
        while True:
            per_request = min(SPOTIFY_MAX_LIMIT, combined_limit - overall_idx)
            if per_request <= 0:
                break
            if PRINT_SEARCH_INFO:
                logging.info("Searching Spotify: q='%s' type=%s limit=%d offset=%d market=%s", q, kind, per_request, offset, MARKET)
            j = spotifysearch(token, q, type_=kind, limit=per_request, offset=offset, market=MARKET)
            if not j:
                break
            items = j.get((kind + "s") if kind in ("album", "track") else "tracks", {}).get("items", [])
            if not isinstance(items, list) or not items:
                break
            for it in items:
                it_id = it.get("id")
                if it_id:
                    key = f"id:{it_id}"
                else:
                    cand_title = _normalize_text_basic(it.get("name"))
                    cand_artists = " ".join(a.get("name", "") for a in it.get("artists", []))
                    cand_artist_norm = _normalize_artist_for_search(cand_artists)
                    album_info = (it.get("album") or {}) if kind == "track" else it
                    cand_album_name = _normalize_title_for_search((album_info.get("name") or ""))
                    key = f"key:{cand_title}|{cand_artist_norm}|{cand_album_name}"
                if key in seen_keys:
                    continue
                overall_idx += 1
                seen_keys.add(key)
                if kind == "track":
                    cand_title = _normalize_text_basic(it.get("name"))
                    cand_artists = " ".join(a.get("name", "") for a in it.get("artists", []))
                    cand_artist_norm = _normalize_artist_for_search(cand_artists)
                    album_info = it.get("album", {}) or {}
                    cand_album_name = _normalize_title_for_search(album_info.get("name"))
                else:
                    cand_title = _normalize_text_basic(it.get("name"))
                    cand_artist_norm = _normalize_text_basic(" ".join(a.get("name", "") for a in it.get("artists", [])))
                    cand_album_name = cand_title
                if PRINT_SEARCH_INFO:
                    logging.info("Candidate #%d: title='%s' | artist='%s' | album='%s'", overall_idx, cand_title, cand_artist_norm, cand_album_name)
                title_ok = _tokens_in_candidate(title_tokens, cand_title)
                artist_ok = (not artist_tokens) or _tokens_in_candidate(artist_tokens, cand_artist_norm) or (remixer_tokens and _tokens_in_candidate(remixer_tokens, cand_artist_norm))
                album_ok = True
                if album_tokens:
                    album_ok = _tokens_in_candidate(album_tokens, cand_album_name)
                accepted = bool(title_ok and artist_ok and album_ok)
                if PRINT_SEARCH_INFO:
                    logging.info("ACCEPTED" if accepted else "REJECTED")
                if accepted:
                    return it
                if overall_idx >= combined_limit:
                    break
            if overall_idx >= combined_limit:
                break
            offset += per_request
            if len(items) < per_request:
                break
        if overall_idx >= combined_limit:
            break

    # Fallback: artist->albums->tracks exploration
    if n_artist:
        artist_search_q = f'artist:"{n_artist}"'
        if PRINT_SEARCH_INFO:
            logging.info("Fallback artist search: %s", artist_search_q)
        artist_resp = spotifysearch(token, artist_search_q, type_="artist", limit=1, offset=0, market=MARKET)
        artist_items = []
        try:
            artist_items = artist_resp.get("artists", {}).get("items", []) if artist_resp else []
        except Exception:
            artist_items = []
        if artist_items:
            artist_id = artist_items[0].get("id")
            if PRINT_SEARCH_INFO:
                logging.info("Found artist id=%s; enumerating albums", artist_id)
            if artist_id:
                a_off = 0
                while True:
                    a_resp = spotify_get_artist_albums(token, artist_id, limit=SPOTIFY_MAX_LIMIT, offset=a_off, market=MARKET)
                    if not a_resp:
                        break
                    albums = a_resp.get("items", []) or []
                    if not albums:
                        break
                    for alb in albums:
                        alb_id = alb.get("id")
                        if not alb_id:
                            continue
                        t_off = 0
                        while True:
                            t_resp = spotify_get_album_tracks(token, alb_id, limit=SPOTIFY_MAX_LIMIT, offset=t_off, market=MARKET)
                            if not t_resp:
                                break
                            tracks = t_resp.get("items", []) or []
                            if not tracks:
                                break
                            for tr in tracks:
                                tr_id = tr.get("id")
                                if tr_id and f"id:{tr_id}" in seen_keys:
                                    continue
                                it_like = {"id": tr.get("id"), "name": tr.get("name"), "artists": tr.get("artists", []), "album": {"name": alb.get("name")}}
                                cand_title = _normalize_text_basic(it_like.get("name"))
                                cand_artists = " ".join(a.get("name", "") for a in it_like.get("artists", []))
                                cand_artist_norm = _normalize_artist_for_search(cand_artists)
                                cand_album_name = _normalize_title_for_search(alb.get("name"))
                                title_ok = _tokens_in_candidate(title_tokens, cand_title)
                                artist_ok = (not artist_tokens) or _tokens_in_candidate(artist_tokens, cand_artist_norm) or (remixer_tokens and _tokens_in_candidate(remixer_tokens, cand_artist_norm))
                                album_ok = True
                                if album_tokens:
                                    album_ok = _tokens_in_candidate(album_tokens, cand_album_name)
                                if title_ok and artist_ok and album_ok:
                                    return it_like
                                seen_keys.add(f"id:{tr_id}" if tr_id else f"key:{cand_title}|{cand_artist_norm}|{cand_album_name}")
                            if len(tracks) < SPOTIFY_MAX_LIMIT:
                                break
                            t_off += SPOTIFY_MAX_LIMIT
                    if len(albums) < SPOTIFY_MAX_LIMIT:
                        break
                    a_off += SPOTIFY_MAX_LIMIT
    return None


# -------------------------
# TAG WRITING / FORMAT-SPECIFIC HANDLERS
# -------------------------

def download_image_bytes(url: str) -> Optional[Tuple[bytes, str]]:
    try:
        r = requests.get(url, timeout=REQUEST_TIMEOUT)
        r.raise_for_status()
        mime = r.headers.get("Content-Type", "") or "image/jpeg"
        return r.content, mime
    except Exception:
        return None


def get_artist_genres(token: str, artist_id: str) -> List[str]:
    try:
        headers = {"Authorization": f"Bearer {token}"}
        r = requests.get(SPOTIFY_ARTIST_URL.format(artist_id), headers=headers, timeout=REQUEST_TIMEOUT)
        if r.ok:
            j = r.json()
            genres = j.get("genres", [])
            if isinstance(genres, list):
                return genres
    except Exception:
        pass
    return []


def remove_existing_pictures_generic(path: Path, audio_obj) -> None:
    """
    Remove pictures depending on container:
    - For ID3: remove APIC frames
    - For FLAC: remove pictures
    - For WAV with RIFF INFO: skip (not reliably supported)
    """
    ext = path.suffix.lower()
    try:
        if isinstance(audio_obj, ID3):
            try:
                audio_obj.delall("APIC")
            except Exception:
                pass
            return
        if ext == ".flac":
            if hasattr(audio_obj, "clear_pictures"):
                try:
                    audio_obj.clear_pictures()
                    return
                except Exception:
                    pass
            if hasattr(audio_obj, "pictures"):
                try:
                    audio_obj.pictures[:] = []
                    return
                except Exception:
                    pass
    except Exception:
        pass


def set_genre_on_audio(path: Path, audio_tmp, genres_list: List[str]) -> None:
    """
    Write or remove genre in format-appropriate manner.
    """
    ext = path.suffix.lower()
    genre_value = "; ".join(genres_list) if genres_list else None
    try:
        if isinstance(audio_tmp, ID3):
            if genre_value:
                try:
                    audio_tmp.delall("TCON")
                except Exception:
                    pass
                audio_tmp.add(TCON(encoding=3, text=genre_value))
            else:
                try:
                    audio_tmp.delall("TCON")
                except Exception:
                    pass
            return
        if ext == ".flac":
            if audio_tmp.tags is None:
                audio_tmp.tags = {}
            if genre_value:
                audio_tmp.tags["genre"] = [genre_value]
            else:
                for k in ("genre", "genres"):
                    if k in audio_tmp.tags:
                        del audio_tmp.tags[k]
            return
        if ext == ".wav":
            if getattr(audio_tmp, "tags", None) is None:
                audio_tmp.tags = {}
            keys_to_try = ["IGEN", "IGNR", "GENR", "GENRE"]
            if genre_value:
                for k in keys_to_try:
                    try:
                        audio_tmp.tags[k] = [genre_value]
                        break
                    except Exception:
                        continue
            else:
                for k in keys_to_try:
                    try:
                        if k in audio_tmp.tags:
                            del audio_tmp.tags[k]
                    except Exception:
                        pass
            return
        if hasattr(audio_tmp, "tags"):
            if audio_tmp.tags is None:
                audio_tmp.tags = {}
            if genre_value:
                audio_tmp.tags["genre"] = [genre_value]
            else:
                try:
                    if "genre" in audio_tmp.tags:
                        del audio_tmp.tags["genre"]
                except Exception:
                    pass
    except Exception:
        pass


def add_picture_to_audio(path: Path, audio_tmp, image_bytes: bytes, mime: str) -> None:
    """
    Add cover art where supported (FLAC, ID3). Skip WAV (not reliable).
    """
    ext = path.suffix.lower()
    try:
        if isinstance(audio_tmp, ID3):
            try:
                audio_tmp.delall("APIC")
            except Exception:
                pass
            try:
                apic = APIC(encoding=3, mime=mime, type=3, desc="Cover", data=image_bytes)
                audio_tmp.add(apic)
            except Exception:
                pass
            return
        if ext == ".flac":
            pic = Picture()
            pic.data = image_bytes
            pic.type = 3
            pic.mime = mime
            try:
                audio_tmp.add_picture(pic)
            except Exception:
                pass
            return
        logging.info("Skipping embedding cover art for WAV file: %s (not reliably supported)", path.name)
    except Exception:
        pass


# -------------------------
# CORE: update metadata for a single file (handles FLAC/MP3/WAV)
# -------------------------

def overwrite_metadata_with_spotify(file_path: Path, token: str) -> bool:
    """
    Core update function:
    - Detect format and open appropriate tag container (FLAC, ID3, WAVE).
    - Read artist/title/album or infer from filename.
    - Allow searches if at least title or artist exists (not require both).
    - Use ISRC-first, then containment-based searches (spotify_find_best_match).
    - Create temp copy, write tags, send original to trash, move temp into original path.
    """
    ext = file_path.suffix.lower()
    wav_has_id3 = False
    audio = None
    try:
        if ext == ".flac":
            audio = FLAC(str(file_path))
        elif ext == ".mp3":
            try:
                audio = ID3(str(file_path))
            except ID3NoHeaderError:
                audio = ID3()
        elif ext == ".wav":
            # Prefer ID3 chunk inside WAV if present
            try:
                audio = ID3(str(file_path))
                wav_has_id3 = True
            except ID3NoHeaderError:
                audio = WAVE(str(file_path))
                wav_has_id3 = False
        else:
            logging.info("Unsupported format: %s", file_path.name)
            return False
    except Exception as e:
        logging.error("Could not open %s: %s", file_path.name, e)
        return False

    # Generic tag reader function based on container type
    tags = None
    try:
        if isinstance(audio, FLAC):
            tags = audio.tags or {}
        elif isinstance(audio, ID3):
            tags = audio
        else:
            tags = getattr(audio, "tags", {}) or {}
    except Exception:
        tags = {}

    def first_tag_generic(k):
        """
        Read a tag generically across FLAC, ID3, and WAVE (RIFF INFO) fallback keys.
        """
        try:
            if isinstance(audio, FLAC):
                v = tags.get(k)
                if v and isinstance(v, (list, tuple)):
                    return str(v[0])
                return str(v) if v else None
            if isinstance(audio, ID3):
                map_frames = {"artist":"TPE1","albumartist":"TPE2","album":"TALB","title":"TIT2","date":"TDRC","tracknumber":"TRCK","discnumber":"TPOS","isrc":"TSRC","genre":"TCON"}
                frame = map_frames.get(k)
                if frame and frame in tags:
                    f = tags.getall(frame)
                    if f:
                        try:
                            txt = f[0].text
                            if isinstance(txt,(list,tuple)):
                                return str(txt[0])
                            return str(txt)
                        except Exception:
                            try:
                                return str(f[0])
                            except Exception:
                                return None
                return None
            # WAVE / RIFF INFO fallback
            if getattr(tags, "get", None):
                v = tags.get(k)
                if v:
                    if isinstance(v,(list,tuple)):
                        return str(v[0])
                    return str(v)
            alt_keys = {
                "title":["INAM","NAME","TITLE"],
                "artist":["IART","AUTH","ARTIST"],
                "album":["IPRD","ALBUM"],
                "date":["ICRD","DATE","YEAR"],
                "tracknumber":["ITRK","TRACKNUMBER"],
                "discnumber":["TPOS","DISCNUMBER"],
                "isrc":["TSRC","ISRC"],
                "genre":["IGEN","IGNR","GENR","GENRE"]
            }
            for alt in alt_keys.get(k, []):
                try:
                    vv = tags.get(alt)
                    if vv:
                        if isinstance(vv,(list,tuple)):
                            return str(vv[0])
                        return str(vv)
                except Exception:
                    continue
            return None
        except Exception:
            return None

    artist = first_tag_generic("artist") or first_tag_generic("albumartist")
    album = first_tag_generic("album")
    title = first_tag_generic("title")
    isrc_tag = first_tag_generic("isrc") or first_tag_generic("ISRC")

    # If either artist or title missing, try filename fallback.
    ai, ti = infer_artist_title_from_filename(file_path)
    artist = artist or ai
    title = title or ti

    # If neither artist nor title are available after fallback, skip.
    if not artist and not title:
        logging.info("Insufficient metadata for: %s", file_path.name)
        return False

    # Log which fields we will use for searching
    if PRINT_SEARCH_INFO:
        if artist and title:
            logging.info("Searching with artist+title (both available).")
        elif title and not artist:
            logging.info("Searching with title only.")
        elif artist and not title:
            logging.info("Searching with artist only.")

    artist_for_search = _strip_parentheses_with_feat(artist) if artist else None
    title_for_search = _strip_parentheses_with_feat(title) if title else None
    album_for_search = _strip_parentheses_with_feat(album) if album else None

    if PRINT_SEARCH_INFO:
        sanitized_artist = _normalize_artist_for_search(artist_for_search) if artist_for_search else ""
        sanitized_title = _normalize_title_for_search(title_for_search) if title_for_search else ""
        sanitized_album = _normalize_title_for_search(album_for_search) if album_for_search else ""
        logging.info("Search input (sanitized): artist='%s' | title='%s' | album='%s' | ext=%s", sanitized_artist, sanitized_title, sanitized_album, ext)

    match = None
    # 1) ISRC-first if available
    if isrc_tag:
        isrc_q = f'isrc:"{isrc_tag.strip()}"'
        if PRINT_SEARCH_INFO:
            logging.info("Attempting ISRC search: %s", isrc_q)
        try:
            j = spotifysearch(token, isrc_q, type_="track", limit=1, offset=0, market=MARKET)
            if j:
                items = j.get("tracks", {}).get("items", [])
                if items:
                    match = items[0]
        except Exception:
            match = None

    # 2) containment/fielded search -- spotify_find_best_match tolerates empty artist or title
    if not match:
        match = spotify_find_best_match(token, artist_for_search, album_for_search, title_for_search, combined_limit=SEARCH_CANDIDATE_LIMIT)

    if not match:
        logging.info("No Spotify match for: %s", file_path.name)
        return False

    # Build metadata fields from match (same extraction used previously)
    meta_artist = None
    meta_title = None
    meta_album = None
    meta_date = None
    meta_track = None
    meta_disc = None
    image_url = None
    artist_id = None
    genres_list: List[str] = []

    if "album" in match and "name" in match:
        meta_title = match.get("name")
        album_info = match.get("album", {})
        meta_album = album_info.get("name")
        artists = match.get("artists", [])
        if artists:
            meta_artist = artists[0].get("name")
            artist_id = artists[0].get("id")
        meta_track = str(match.get("track_number")) if match.get("track_number") else None
        meta_disc = str(match.get("disc_number")) if match.get("disc_number") else None
        images = album_info.get("images", [])
        if images:
            image_url = images[0].get("url")
        meta_date = album_info.get("release_date")
        if isinstance(album_info.get("genres"), list) and album_info.get("genres"):
            genres_list = album_info.get("genres", [])
    else:
        meta_album = match.get("name")
        artists = match.get("artists", [])
        if artists:
            meta_artist = artists[0].get("name")
            artist_id = artists[0].get("id")
        images = match.get("images", [])
        if images:
            image_url = images[0].get("url")
        meta_date = match.get("release_date")
        if isinstance(match.get("genres"), list) and match.get("genres"):
            genres_list = match.get("genres", [])

    if PRINT_SEARCH_INFO:
        candidate_album = None
        if "album" in match:
            candidate_album = (match.get("album") or {}).get("name")
        else:
            candidate_album = match.get("name")
        logging.info("Spotify candidate: spotify_title='%s' | spotify_artist='%s' | spotify_album='%s'", meta_title, meta_artist, candidate_album)

    # Optionally fetch artist genres
    if artist_id:
        try:
            g = get_artist_genres(token, artist_id)
            if g:
                genres_list = g
        except Exception:
            pass

    if not genres_list and meta_artist:
        try:
            j = spotifysearch(token, f'artist:"{meta_artist}"', type_="artist", limit=1, offset=0, market=MARKET)
            if j:
                items = j.get("artists", {}).get("items", [])
                if items:
                    gg = items[0].get("genres", [])
                    if isinstance(gg, list) and gg:
                        genres_list = gg
        except Exception:
            pass

    # Create a temp copy, edit temp, send original to trash, move temp into original path
    temp_path = None
    try:
        temp_path = unique_temp_copy(file_path)

        # open temp with appropriate handler (preserve WAV-ID3 behavior)
        if ext == ".flac":
            audio_tmp = FLAC(str(temp_path))
        elif ext == ".mp3":
            try:
                audio_tmp = ID3(str(temp_path))
            except ID3NoHeaderError:
                audio_tmp = ID3()
        elif ext == ".wav":
            if wav_has_id3:
                try:
                    audio_tmp = ID3(str(temp_path))
                except ID3NoHeaderError:
                    audio_tmp = ID3()
            else:
                audio_tmp = WAVE(str(temp_path))
                if getattr(audio_tmp, "tags", None) is None:
                    audio_tmp.tags = {}
        else:
            logging.info("Unsupported for write: %s", file_path.name)
            return False

        # If only updating genre, do that and exit
        if UPDATE_ONLY_GENRE:
            set_genre_on_audio(file_path, audio_tmp, genres_list)
            try:
                if isinstance(audio_tmp, ID3):
                    audio_tmp.save(str(temp_path))
                else:
                    audio_tmp.save()
            except Exception:
                try:
                    audio_tmp.save(str(temp_path))
                except Exception:
                    pass
            send_original_to_trash(file_path)
            shutil.move(str(temp_path), str(file_path))
            return True

        # Generic setter for tags across formats
        def set_tag_general(key: str, value: Optional[str]):
            if value is None:
                return
            try:
                if isinstance(audio_tmp, FLAC):
                    if audio_tmp.tags is None:
                        audio_tmp.tags = {}
                    audio_tmp.tags[key] = [value.strip()]
                    return
                if isinstance(audio_tmp, ID3):
                    if key == "title":
                        audio_tmp.delall("TIT2"); audio_tmp.add(TIT2(encoding=3, text=value.strip()))
                    elif key == "artist":
                        audio_tmp.delall("TPE1"); audio_tmp.add(TPE1(encoding=3, text=value.strip()))
                    elif key == "album":
                        audio_tmp.delall("TALB"); audio_tmp.add(TALB(encoding=3, text=value.strip()))
                    elif key == "date":
                        audio_tmp.delall("TDRC"); audio_tmp.add(TDRC(encoding=3, text=value.strip()))
                    elif key == "tracknumber":
                        audio_tmp.delall("TRCK"); audio_tmp.add(TRCK(encoding=3, text=value.strip()))
                    elif key == "discnumber":
                        audio_tmp.delall("TPOS"); audio_tmp.add(TPOS(encoding=3, text=value.strip()))
                    elif key == "genre":
                        audio_tmp.delall("TCON"); audio_tmp.add(TCON(encoding=3, text=value.strip()))
                    return
                if ext == ".wav":
                    if getattr(audio_tmp, "tags", None) is None:
                        audio_tmp.tags = {}
                    info_map = {
                        "title":["INAM","NAME","TITLE"],
                        "artist":["IART","AUTH","ARTIST"],
                        "album":["IPRD","ALBUM"],
                        "date":["ICRD","DATE","YEAR"],
                        "tracknumber":["ITRK","TRACKNUMBER"],
                        "discnumber":["TPOS","DISCNUMBER"],
                        "genre":["IGEN","IGNR","GENR","GENRE"]
                    }
                    keys = info_map.get(key, [key.upper()])
                    for k in keys:
                        try:
                            audio_tmp.tags[k] = [value.strip()]
                            break
                        except Exception:
                            continue
            except Exception:
                pass

        # Always update genres on the temp object (add or remove)
        set_genre_on_audio(file_path, audio_tmp, genres_list)

        # Update cover art if available
        remove_existing_pictures_generic(file_path, audio_tmp)
        if image_url:
            got = download_image_bytes(image_url)
            if got:
                image_bytes, mime = got
                add_picture_to_audio(file_path, audio_tmp, image_bytes, mime)

        # Update title/artist/album only if allowed (we never touch albumartist)
        if OVERWRITE_TITLE_ARTIST_OR_ALBUM:
            set_tag_general("title", meta_title or title or "")
            set_tag_general("artist", meta_artist or artist or "")
            if meta_album:
                set_tag_general("album", meta_album or album or None)

        # Update date/track/disc if present
        if meta_date:
            set_tag_general("date", meta_date)
        if meta_track:
            set_tag_general("tracknumber", meta_track)
        if meta_disc:
            set_tag_general("discnumber", meta_disc)

        # Save the temp file's tags
        try:
            if isinstance(audio_tmp, ID3):
                audio_tmp.save(str(temp_path))
            else:
                audio_tmp.save()
        except Exception:
            try:
                audio_tmp.save(str(temp_path))
            except Exception:
                pass

        # Replace original: send original to trash and move temp into original path
        send_original_to_trash(file_path)
        shutil.move(str(temp_path), str(file_path))
        return True

    except Exception as e:
        if temp_path and temp_path.exists():
            try:
                temp_path.unlink()
            except Exception:
                pass
        logging.error("Failed updating %s: %s", file_path.name, e)
        return False


# -------------------------
# FILE ITERATION + MAIN
# -------------------------

def iter_audio_files(root: Path, recursive: bool):
    """
    Yield audio files with supported extensions.
    """
    patterns = ("*.flac", "*.mp3", "*.wav")
    if recursive:
        for pat in patterns:
            yield from root.rglob(pat)
    else:
        for pat in patterns:
            yield from root.glob(pat)


def get_creation_time(path: Path) -> float:
    """
    Return the creation time (birth time) for a file, falling back to platform-appropriate values:
    - If stat result has st_birthtime (macOS, some BSDs), return that.
    - On Windows, use st_ctime (typically creation time).
    - Otherwise fall back to st_mtime (modification time) because many Linux filesystems do not expose birthtime.
    Returns a float epoch seconds; returns 0.0 on error.
    """
    try:
        s = path.stat()
        # Prefer st_birthtime if available (macOS, some filesystems)
        if hasattr(s, "st_birthtime"):
            return float(s.st_birthtime)
        # On Windows, st_ctime is typically the creation time
        if platform.system() == "Windows":
            return float(s.st_ctime)
        # Fallback: use st_mtime when birthtime is not available
        return float(s.st_mtime)
    except Exception:
        return 0.0


def main():
    # Load credentials (client id/secret) and music path from credentials.json
    try:
        with CREDENTIALS_PATH.open("r", encoding="utf-8") as f:
            data = json.load(f)
        client_id = str(data.get("client_id", "")).strip()
        client_secret = str(data.get("client_secret", "")).strip()
        music_path = data.get("music_path") or data.get("source_dir") or data.get("music_dir")
        if not client_id or not client_secret:
            logging.error("Missing client_id or client_secret in credentials.json")
            return
        if not music_path:
            logging.error("Missing music_path in credentials.json")
            return
        SOURCE_DIR = Path(music_path)
        if not SOURCE_DIR.exists() or not SOURCE_DIR.is_dir():
            logging.error("music_path from credentials.json is not a valid directory: %s", SOURCE_DIR)
            return
        logging.info("Loaded music path from credentials.json: %s", SOURCE_DIR)

        token, expires_at = get_spotify_token(client_id, client_secret)
        logging.info("Spotify token obtained")
    except Exception as e:
        logging.error("Failed to load credentials or obtain token: %s", e)
        return

    # Gather files
    paths = [p for p in iter_audio_files(SOURCE_DIR, RECURSIVE)]
    # Sort by creation date (birth time) where available; fallback to mtime when not.
    paths.sort(key=lambda p: get_creation_time(p) if p.exists() else 0, reverse=True)
    total = len(paths)
    logging.info("Found %d audio files (FLAC/MP3/WAV) in %s", total, SOURCE_DIR)

    if PROCESS_TOP_X and isinstance(PROCESS_TOP_X, int) and PROCESS_TOP_X > 0:
        limit = min(PROCESS_TOP_X, total)
        paths = paths[:limit]

    updated = skipped = failed = 0
    for i, path in enumerate(paths, 1):
        try:
            if int(time.time()) >= expires_at:
                try:
                    token, expires_at = get_spotify_token(client_id, client_secret)
                except Exception:
                    logging.error("Failed to refresh Spotify token")
                    break

            logging.info("Processing (%d/%d): %s", i, len(paths), path.name)
            ok = overwrite_metadata_with_spotify(path, token)
            if ok:
                logging.info("Updated metadata for: %s (original moved to trash)", path.name)
                updated += 1
            else:
                skipped += 1

        except KeyboardInterrupt:
            break
        except Exception as e:
            logging.error("Unexpected error processing %s: %s", path.name, e)
            failed += 1

    logging.info("Completed. Updated: %d, Skipped: %d, Failed: %d, Total found: %d", updated, skipped, failed, total)


if __name__ == "__main__":
    main()